# Validation of White Matter and Out Contacts Algorithm

2/7/19: Since we are interested in segmenting contacts in SEEG based on parcellation of white matter and brain mask.

Inputs:
- brain mask in native MRI space
- white matter mask in native MRI space
- contact coords in native MRI space

Output:
- list of contacts in white matter
- list of contacts in out

In [8]:
import numpy as np
import scipy
import os
import sys
import nibabel as nb
import json

sys.path.append('../../')

%matplotlib inline
# import basic plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

# Import magic commands for jupyter notebook 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set Data Directories

In [10]:
datadir = "/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/outputfiles/"
patid = "la04"

patdir = os.path.join(datadir, patid, "tvb", "native-space")
nativedir = os.path.join(datadir, patid, "nativespace-transform")
coregistrationdir = os.path.join(datadir, patid, "coregistration")

print(os.listdir(nativedir))
print(os.listdir(patdir))

['lh_native.pial', 'lh_native.asc', 'rh_native.pial', 'aparc.a2009s+aseg.native.mgz', 'wm_native.mgz', 'register.native.dat', 'norm.mgz', 'register.conformed.dat', 'aparc+aseg.native.mgz', 'desikan_native.mgz', 'aseg.mgz', 'rh_native.asc', 'destrieux_native.mgz', 'brainmask_native.mgz', 'brainmask_native_inct.nii.gz', 'brainmask_native.nii.gz', 'fsl_brainmask_to_ct_omat.txt']
['connectivity_dk.zip', 'surface_cort.zip', 'connectivity_destrieux.zip', 'surface_subcort.zip', 'region_mapping_cort.txt', 'region_mapping_subcort.txt']


In [12]:
brainmask_file = os.path.join(nativedir, "brainmask_native.nii.gz")
wmmask_file = os.path.join(nativedir, "wm_native.mgz")
contactcoords_file = os.path.join(coregistrationdir, "{}_native_flirt_elec_xyz.txt".format(patid))

# Load Data

# Define Algorithm to Map Contacts to Outside